In [6]:
%pip install statsmodels
%pip install linearmodels

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.iv import IV2SLS
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# Load the dataset
file_path = r'C:\Users\Davis\Downloads\paper.dta'
df = pd.read_stata(file_path)


# Assuming 'df' is your DataFrame loaded with the necessary data

# Part I: Generate market shares and C4 for LIS Weights for BASIC PDP
df['true_weight_check_cw2'] = np.where((df['pdp'] == 1) & (df['basic'] == 1), df['true_weight_check_cw'], np.nan)
df.sort_values(by='true_weight_check_cw2', ascending=False, inplace=True)
df['idd'] = df.groupby('new_id1').cumcount() + 1

for i in range(1, 5):
    df[f'C{i}_plan_weight2'] = (df.groupby('new_id1')['idd'].transform(lambda x: x == i)).astype(int)
    df[f'C{i}_weight2'] = df['true_weight_check_cw2'] * df[f'C{i}_plan_weight2']

df['conc_weig_basicpdpd_c4'] = sum(df[f'C{i}_weight2'] for i in range(1, 5))
df.drop(columns=['idd'] + [f'C{i}_weight2' for i in range(1, 5)] + [f'C{i}_plan_weight2' for i in range(1, 5)], inplace=True)

# Part II: Controls - Generate HHI by Firm
df['enrol_tot_share'] = np.where((df['enrol_tot_num'] != 0) & pd.notnull(df['enrol_tot_num']) & (df['enrol_tot_denominator'] != 0) & pd.notnull(df['enrol_tot_denominator']),
                                 df['enrol_tot_num'] / df['enrol_tot_denominator'], 0)
df['firm_tot_share'] = df.groupby(['new_id1', 'Parent_Organization'])['enrol_tot_share'].transform('sum')
df['iddd'] = df.groupby(['new_id1', 'Parent_Organization']).cumcount() + 1
df['temp2'] = (df.groupby('new_id1')['iddd'].transform('min') == df['iddd']).astype(int)
df['num_firms'] = df.groupby('new_id1')['temp2'].transform('sum')
df['firm_tot_share2'] = np.where(df['temp2'] == 1, df['firm_tot_share'], np.nan)
df['hershf_firm_tot'] = df.groupby('new_id1')['firm_tot_share2'].transform(lambda x: ((x ** 2).sum() - (1 / df['num_firms'])) / (1 - (1 / df['num_firms'])))
df.drop(columns=['iddd', 'temp2', 'firm_tot_share2'], inplace=True)

# Get the sample right before proceeding
df.drop_duplicates(subset=['regionid', 'year'], keep='last', inplace=True)
df.dropna(subset=['regionid', 'year'], inplace=True)

# Part III: Dependent Variable and Other Covariates
df.sort_values(by=['regionid', 'year'], inplace=True)
df['logmean_w_reg_bpr'] = np.log(df['mean_w_reg_basprem'])
df['logmean_w_reg_bpr_ch'] = df.groupby('regionid')['logmean_w_reg_bpr'].diff()

df['change'] = df['year'] > 2008
df['laghershf_firm_tot'] = df.groupby('regionid')['hershf_firm_tot'].shift(1)
df['lagunemploymentrate'] = df.groupby('regionid')['unemploymentrate'].shift(1)

# Dropping year == 2006 for regression analysis
df_filtered = df[df['year'] != 2006]


# Generate a binary variable for year 2009
df['year2009'] = np.where(df['year'] > 2008, 1, 0)

# Create interaction term for year2009 and mapd_regio_2006share
df['regio_2006share2'] = df['mapd_regio_2006share'] * df['year2009']

# Compute the change in the log of mean weighted reg bpr
df['logmean_w_reg_bpr'] = np.log(df['mean_w_reg_basprem'])
df.sort_values(by=['regionid', 'year'], inplace=True)
df['logmean_w_reg_bpr_ch'] = df.groupby('regionid')['logmean_w_reg_bpr'].shift(-1) - df['logmean_w_reg_bpr']

# Add lagged variables
df['laghershf_firm_tot'] = df.groupby('regionid')['hershf_firm_tot'].shift(1)
df['lagunemploymentrate'] = df.groupby('regionid')['unemploymentrate'].shift(1)

def run_regression(df, dependent, independent, controls, interaction_terms=None):
    formula = f"{dependent} ~ {' + '.join([independent] + controls)}"
    if interaction_terms:
        for term in interaction_terms:
            formula += f" * {term}"
    model = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['regionid']})
    return model

# Example controls from the Stata code (simplified for demonstration)
control_sets = [
    ["conc_weig_basicpdpd_c4"],
    ["conc_weig_basicpdpd_c4"] + [f"ttrend{i}" for i in range(1, 35)],
    ["conc_weig_basicpdpd_c4", "laghershf_firm_tot", "lagunemploymentrate"],
    ["conc_weig_basicpdpd_c4", "laghershf_firm_tot", "lagunemploymentrate"] + [f"ttrend{i}" for i in range(1, 35)],
    ["conc_weig_basicpdpd_c4", "laghershf_firm_tot",  "lagunemploymentrate", "mean_weig_reg_vintage" "mean_weig_reg_pharmacies", "mean_weig_reg_drugs"],
    ["conc_weig_basicpdpd_c4", "laghershf_firm_tot",  "lagunemploymentrate", "mean_weig_reg_vintage" "mean_weig_reg_pharmacies", "mean_weig_reg_drugs"] + [f"ttrend{i}" for i in range(1, 35)]
]

# Run regressions for each set of controls and print summaries
for index, controls in enumerate(control_sets, start=1):
    model = run_regression(df, "logmean_w_reg_bpr_ch", "conc_weig_basicpdpd_c4", controls)
    print(f"Model {index} Summary:\n", model.summary(), "\n\n")

def run_iv_regression(df, dependent, endog, exog, instrument):
    iv_formula = f"{dependent} ~ 1 + {exog} + [{endog} ~ {instrument}]"
    iv_model = IV2SLS.from_formula(iv_formula, data=df).fit(cov_type='clustered', clusters=df['regionid'])
    return iv_model

# Example IV regression with specified controls
iv_controls = ["laghershf_firm_tot", "lagunemploymentrate"]  # Simplified for demonstration
iv_model = run_iv_regression(df, "logmean_w_reg_bpr_ch", "conc_weig_basicpdpd_c4", " + ".join(iv_controls), "regio_2006share2")
print("IV Model Summary:\n", iv_model.summary)



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\Davis\AppData\Local\Temp\ipykernel_14668\371290269.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['true_weight_check_cw2'] = np.where((df['pdp'] == 1) & (df['basic'] == 1), df['true_weight_check_cw'], np.nan)
C:\Users\Davis\AppData\Local\Temp\ipykernel_14668\371290269.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['idd'] = df.groupby('new_id1').cumcount() + 1
C:\Users\Davis\AppData\Local\Temp\ipykernel_14668\371290269.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

ValueError: zero-size array to reduction operation maximum which has no identity